<a href="https://colab.research.google.com/github/AlexHarry17/Reinforcement_Learning/blob/master/Deep_Learning_Course/Cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sourced code to learn reinforcement learning from https://github.com/simoninithomas/Deep_reinforcement_learning_Course/blob/master/Policy%20Gradients/Cartpole/Cartpole%20REINFORCE%20Monte%20Carlo%20Policy%20Gradients.ipynb

In [0]:
import tensorflow as tf
import numpy as np
import gym

In [76]:
env = gym.make('CartPole-v0')
env = env.unwrapped
env.seed(1) # Seed is to help with reproducability

[1]

In [0]:
#hyper params
state_size = 4
action_size=env.action_space.n

max_episodes=1000
learning_rate=0.01
gamma=0.95

In [0]:
def discount_normalize_rewards(episode_rewards):
  discounted_episode_rewards = np.zeros_like(episode_rewards)
  cumulative = 0.0
  for i in reversed(range(len(episode_rewards))):
    cumulative = cumulative*gamma*episode_rewards[i]
    discounted_episode_rewards[i]=cumulative
  mean=np.mean(discounted_episode_rewards)
  std=np.std(discounted_episode_rewards)
  discounted_episode_rewards=(discounted_episode_rewards-mean)/(std)
  return discounted_episode_rewards

In [0]:
with tf.name_scope('inputs'):
  input_=tf.placeholder(tf.float32, [None, state_size], name='input_')
  actions=tf.placeholder(tf.int32, [None, action_size], name='actions')
  discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,], name='discounted_episode_rewards')

  mean_reward_=tf.placeholder(tf.float32, name='mean_reward')

# Create tensorflow layers

  with tf.name_scope("fc1"):
    fc1 = tf.contrib.layers.fully_connected(inputs=input_, num_outputs=action_size, activation_fn=tf.nn.relu, weights_initializer=tf.contrib.layers.xavier_initializer())
  
  with tf.name_scope("fc2"):
    fc2 = tf.contrib.layers.fully_connected(inputs=fc1, num_outputs=action_size, activation_fn=tf.nn.relu, weights_initializer=tf.contrib.layers.xavier_initializer())

  with tf.name_scope("fc3"):
    fc3 = tf.contrib.layers.fully_connected(inputs=fc2, num_outputs=action_size, activation_fn=None, weights_initializer=tf.contrib.layers.xavier_initializer())
  
  with tf.name_scope("softmax"):
    action_distribution=tf.nn.softmax(fc3)
  
  with tf.name_scope("loss"):
    neg_log_prob=tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc3, labels=actions)
    loss = tf.reduce_mean(neg_log_prob*discounted_episode_rewards)

  with tf.name_scope("train"):
    train_opt=tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [0]:
# TensorBoard

writer = tf.summary.FileWriter("/tensorboard/pg/1")

tf.summary.scalar("Loss", loss)

write_op=tf.summary.merge_all()

In [81]:
allRewards= []
total_rewards = 0
maximumRewardRecorded=0
episode=0
episode_states, episode_actions, episode_rewards = [],[],[]
saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for episode in range(max_episodes):
    episode_rewards_sum = 0

    state = env.reset()

    # env.render()

    while True:
      action_propbability_distribution = sess.run(action_distribution, feed_dict={input_:state.reshape([1,4])})

      action = np.random.choice(range(action_propbability_distribution.shape[1]), p=action_propbability_distribution.ravel())

      # Perform action
      new_state, reward, done, info = env.step(action)

      episode_states.append(state)

      action_ = np.zeros(action_size)
      action_[action] = 1

      episode_actions.append(reward)
      episode_rewards.append(reward)

      if done:
        #calc sum reward

        episode_rewards_sum = np.sum(episode_rewards)

        allRewards.append(episode_rewards_sum)

        total_rewards = np.sum(allRewards)

        mean_reward = np.divide(total_rewards, episode+1)

        maximumRewardRecorded = np.amax(allRewards)


        print("==========================================")
        print("Episode: ", episode)
        print("Reward: ", episode_rewards_sum)
        print("Mean Reward", mean_reward)
        print("Max reward so far: ", maximumRewardRecorded)

        discounted_episode_rewards = discount_normalize_rewards(episode_rewards)

        #         # Feedforward, gradient and backpropagation
        # loss_, _ = sess.run([loss, train_opt], feed_dict={input_: np.vstack(np.array(episode_states)),
        #                                                   actions: np.vstack(np.array(episode_actions)),
        #                                                   discounted_episode_rewards_: discounted_episode_rewards 
        #                                                 })
        
        #         # Write TF Summaries
        # summary = sess.run(write_op, feed_dict={input_: np.vstack(np.array(episode_states)),
        #                                                   actions: np.vstack(np.array(episode_actions)),
        #                                                   discounted_episode_rewards_: discounted_episode_rewards,
        #                                                     mean_reward_: mean_reward
        #                                                 })
        
        # writer.add_summary(summary, episode)
        # writer.flush()

        episode_states, episode_actions, episode_rewards = [],[],[]
        break

      if episode % 100 == 0:
        saver.save(sess, "./models/model.ckpt")
        # print("Model saved")
        




Episode:  0
Reward:  20.0
Mean Reward 20.0
Max reward so far:  20.0
Episode:  1
Reward:  53.0
Mean Reward 36.5
Max reward so far:  53.0
Episode:  2
Reward:  14.0
Mean Reward 29.0
Max reward so far:  53.0
Episode:  3
Reward:  53.0
Mean Reward 35.0
Max reward so far:  53.0
Episode:  4
Reward:  40.0
Mean Reward 36.0
Max reward so far:  53.0
Episode:  5
Reward:  10.0
Mean Reward 31.666666666666668
Max reward so far:  53.0
Episode:  6
Reward:  16.0
Mean Reward 29.428571428571427
Max reward so far:  53.0
Episode:  7
Reward:  12.0
Mean Reward 27.25
Max reward so far:  53.0
Episode:  8
Reward:  30.0
Mean Reward 27.555555555555557
Max reward so far:  53.0
Episode:  9
Reward:  13.0
Mean Reward 26.1
Max reward so far:  53.0
Episode:  10
Reward:  19.0
Mean Reward 25.454545454545453
Max reward so far:  53.0
Episode:  11
Reward:  14.0
Mean Reward 24.5
Max reward so far:  53.0
Episode:  12
Reward:  17.0
Mean Reward 23.923076923076923
Max reward so far:  53.0
Episode:  13
Reward:  16.0
Mean Reward 23.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


Episode:  17
Reward:  15.0
Mean Reward 21.5
Max reward so far:  53.0
Episode:  18
Reward:  34.0
Mean Reward 22.157894736842106
Max reward so far:  53.0
Episode:  19
Reward:  19.0
Mean Reward 22.0
Max reward so far:  53.0
Episode:  20
Reward:  17.0
Mean Reward 21.761904761904763
Max reward so far:  53.0
Episode:  21
Reward:  19.0
Mean Reward 21.636363636363637
Max reward so far:  53.0
Episode:  22
Reward:  16.0
Mean Reward 21.391304347826086
Max reward so far:  53.0
Episode:  23
Reward:  11.0
Mean Reward 20.958333333333332
Max reward so far:  53.0
Episode:  24
Reward:  23.0
Mean Reward 21.04
Max reward so far:  53.0
Episode:  25
Reward:  26.0
Mean Reward 21.23076923076923
Max reward so far:  53.0
Episode:  26
Reward:  19.0
Mean Reward 21.14814814814815
Max reward so far:  53.0
Episode:  27
Reward:  39.0
Mean Reward 21.785714285714285
Max reward so far:  53.0
Episode:  28
Reward:  30.0
Mean Reward 22.06896551724138
Max reward so far:  53.0
Episode:  29
Reward:  16.0
Mean Reward 21.866666

In [82]:
#Play cartpole

with tf.Session() as sess:
    env.reset()
    rewards = []
    
    # Load the model
    saver.restore(sess, "./models/model.ckpt")

    for episode in range(10):
        state = env.reset()
        step = 0
        done = False
        total_rewards = 0
        print("****************************************************")
        print("EPISODE ", episode)

        while True:
            

            # Choose action a, remember WE'RE NOT IN A DETERMINISTIC ENVIRONMENT, WE'RE OUTPUT PROBABILITIES.
            action_probability_distribution = sess.run(action_distribution, feed_dict={input_: state.reshape([1,4])})
            #print(action_probability_distribution)
            action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob


            new_state, reward, done, info = env.step(action)

            total_rewards += reward

            if done:
                rewards.append(total_rewards)
                print ("Score", total_rewards)
                break
            state = new_state
    env.close()
    print ("Score over time: " +  str(sum(rewards)/10))

****************************************************
EPISODE  0
Score 11.0
****************************************************
EPISODE  1
Score 32.0
****************************************************
EPISODE  2
Score 21.0
****************************************************
EPISODE  3
Score 22.0
****************************************************
EPISODE  4
Score 24.0
****************************************************
EPISODE  5
Score 16.0
****************************************************
EPISODE  6
Score 19.0
****************************************************
EPISODE  7
Score 13.0
****************************************************
EPISODE  8
Score 20.0
****************************************************
EPISODE  9
Score 12.0
Score over time: 19.0
